In [1]:
import tensorflow as tf
import numpy as np

In [2]:
import pickle
with open ('outfile.txt', 'rb') as fp:
    itemlist = pickle.load(fp)
    
print (len(itemlist[1]))

32


In [3]:
inputs = []
labels = []
for sets in itemlist:
    for item in sets:
        elem = []
        elem.extend(item['state'])
        inputs.append(elem)
        if item['won']:
            labels.append(1.0 * item['move'] / 60.0)
        else:
            labels.append(-1.0 * item['move'] / 60.0)
                          
print(len(labels))

28153


In [4]:
learning_rate = 0.001
training_epochs = 200
batch_size = 256 

n_inputs = 7 * 7 + 3

In [5]:
def fully_conneted_layer(inputs, output_len, name, stddev=0.1):
    W1 = tf.get_variable(name + "_weights", shape=[int(inputs.get_shape()[-1]), output_len], initializer=tf.truncated_normal_initializer(stddev=stddev))
    b1 = tf.get_variable(name + "_bias", shape=[output_len], initializer=tf.constant_initializer(value=0.0))
    return tf.matmul(inputs, W1) + b1

In [6]:
def model(inputs):
    fc1 = fully_conneted_layer(inputs, 512, 'fc_1', stddev = 0.02)
    fc1 = tf.nn.relu(fc1)

    fc2 = fully_conneted_layer(fc1, 256, 'fc_2', stddev = 0.02)
    fc2 = tf.nn.relu(fc2)

    #fc2 = fully_conneted_layer(inputs, 256, 'fc_2', stddev = 0.02)
    #fc2 = tf.nn.relu(fc2)

    out = fully_conneted_layer(fc2, 1, 'fc_3', stddev = 0.02)
    #mout = tf.nn.sigmoid(logits)

    return out

In [7]:
x = tf.placeholder("float", [None, n_inputs])
y = tf.placeholder("float", [None, 1])

In [8]:
mod = model(x)

In [9]:

error = tf.subtract(mod, y)
mse = tf.reduce_mean(tf.square(error))
optimizer = tf.train.AdamOptimizer().minimize(mse)

In [10]:
init = tf.global_variables_initializer()

save_file = './model.ckpt'
saver = tf.train.Saver()    

In [11]:
import math
def batches(batch_size, features, labels):
    assert len(features) == len(labels)
    res = []
    for i in range(0, len(features), batch_size):
        size = min(batch_size, len(features) -  i)
        features_slice = features[i : i + size]
        labels_slice = labels[i : i + size]
        res.append([features_slice, labels_slice])
    return res

In [12]:
print(len(inputs))

with tf.Session() as session:
    session.run(init)
    
    for epoch in range(training_epochs):
        btchs = batches(batch_size, inputs, labels)
        
        for i in range(len(btchs)):
            batch_x, batch_y = btchs[i]
            batch_y = np.array(batch_y).reshape(-1, 1)
            error, _ = session.run([mse, optimizer], feed_dict={x: batch_x, y: batch_y})
            
        if epoch % 5 == 0:
            #valid_accuracy = session.run(accuracy, \
            #    feed_dict={ \
            #        x: mnist.validation.images, \
            #        y: mnist.validation.labels, \
            #               keep_prob: 1}) 
            print('Epoch {:<3} - mse: {}'.format( \
                epoch, \
                error)) \
       
    tf_weights1 = tf.get_collection(tf.GraphKeys.VARIABLES, "fc_1_weights")[0]
    tf_biases1 = tf.get_collection(tf.GraphKeys.VARIABLES, "fc_1_bias")[0]
    tf_weights2 = tf.get_collection(tf.GraphKeys.VARIABLES, "fc_2_weights")[0]
    tf_biases2 = tf.get_collection(tf.GraphKeys.VARIABLES, "fc_2_bias")[0]
    tf_weights3 = tf.get_collection(tf.GraphKeys.VARIABLES, "fc_3_weights")[0]
    tf_biases3 = tf.get_collection(tf.GraphKeys.VARIABLES, "fc_3_bias")[0]
    
    weights1 = tf_weights1.eval()
    biases1 = tf_biases1.eval()
    weights2 = tf_weights2.eval()
    biases2 = tf_biases2.eval()
    weights3 = tf_weights3.eval()
    biases3 = tf_biases3.eval()
 
    np.savetxt('w1.txt', weights1)
    np.savetxt('b1.txt', biases1)
    
    np.savetxt('w2.txt', weights2)
    np.savetxt('b2.txt', biases2)
    
    np.savetxt('w3.txt', weights3)
    np.savetxt('b3.txt', biases3)
   
    saver.save(session, save_file)

28153
Epoch 0   - mse: 0.10381358116865158
Epoch 5   - mse: 0.10062479972839355
Epoch 10  - mse: 0.08790715038776398
Epoch 15  - mse: 0.0806150734424591
Epoch 20  - mse: 0.07693179696798325
Epoch 25  - mse: 0.0758780762553215
Epoch 30  - mse: 0.06891506910324097
Epoch 35  - mse: 0.061274558305740356
Epoch 40  - mse: 0.058182016015052795
Epoch 45  - mse: 0.046853676438331604
Epoch 50  - mse: 0.0413372777402401
Epoch 55  - mse: 0.03810950368642807
Epoch 60  - mse: 0.03185166418552399
Epoch 65  - mse: 0.03141096979379654
Epoch 70  - mse: 0.023607127368450165
Epoch 75  - mse: 0.027650058269500732
Epoch 80  - mse: 0.026803454384207726
Epoch 85  - mse: 0.017680397257208824
Epoch 90  - mse: 0.015137093141674995
Epoch 95  - mse: 0.018694967031478882
Epoch 100 - mse: 0.015158242546021938
Epoch 105 - mse: 0.013815158046782017
Epoch 110 - mse: 0.012605990283191204
Epoch 115 - mse: 0.015602700412273407
Epoch 120 - mse: 0.014956174418330193
Epoch 125 - mse: 0.01083381474018097
Epoch 130 - mse: 0.01

In [13]:
##tf.reset_default_graph()

In [14]:
saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess, save_file)

    test_accuracy = sess.run(
        accuracy,
        feed_dict={x: mnist.test.images, y: mnist.test.labels})

print('Test Accuracy: {}'.format(test_accuracy))

NameError: name 'accuracy' is not defined

In [ ]:
print('Accuracy: {}'.format(accuracy.name))
print('Hiiden input name: {}'.format(weights['hidden_layer'].name))
print('Biases out name: {}'.format(biases['out'].name))